In [1]:
import tweepy
import json
import time
from pandas import Series, DataFrame
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# Authorization Codes
consumer_key = "ZCVjbgfaSGlrJVh7T2cqIFfMR"
consumer_secret = "yah2rFbZE1qX9n1MDaqObT1PfCOSYlvxIyXYuCO4ZrSUfSvjEC"
access_key = "781555439183859712-tS4fKwo03ujDtNL0UibVIXauXRFdFvs"
access_secret = "3YrATCHVvP2atvFs4M6VJ0dFvsdE3BBKoZ2OcvKdqPzJp"

# OAuth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# API Call
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser())

In [2]:
# Test API by pulling data from Donald Trumps twitter
data = api.user_timeline(id='realDonaldTrump', count = 200)  #Example:@realDonaldTrump
for i in range(10):
    print([i],data[i]['text'])

[0] The big story is the "unmasking and surveillance" of people that took place during the Obama Administration.
[1] I will be announcing my decision on Paris Accord, Thursday at 3:00 P.M.  The White House Rose Garden.  MAKE AMERICA GREAT AGAIN!
[2] Crooked Hillary Clinton now blames everybody but herself, refuses to say she was a terrible candidate. Hits Facebook &amp; even Dems &amp; DNC.
[3] It was an honor to welcome the Prime Minister of Vietnam, Nguyễn Xuân Phúc to the @WhiteHouse this afternoon. https://t.co/6YbrIvJUCY
[4] We traveled the world to strengthen long-standing alliances, and to form new partnerships. See more at:… https://t.co/FT4kyPoAl0
[5] I will be announcing my decision on the Paris Accord over the next few days. MAKE AMERICA GREAT AGAIN!
[6] Hopefully Republican Senators, good people all, can quickly get together and pass a new (repeal &amp; replace) HEALTHCARE bill. Add saved $'s.
[7] Kathy Griffin should be ashamed of herself. My children, especially my 11 yea

In [25]:
search_query =  '"TRUMP" -filter:retweets'
data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed')
all_data = list(data.values())[0]


In [26]:
while(len(all_data) <= 1000):
    #time.sleep(5)
    last = all_data[-1]['id']
    data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed', since_id = last)
    all_data += list(data.values())[0][1:]


In [27]:
len(all_data)

1088

In [28]:
tweet = []
number_favourites = []
number_retweet = []
vs_compound = []
vs_pos = []
vs_neu = []
vs_neg = []

analyzer = SentimentIntensityAnalyzer()

for i in range(0, len(all_data)):
    tweet.append(all_data[i]['text'])
    number_retweet.append(all_data[i]['retweet_count'])
    number_favourites.append(all_data[i]['favorite_count'])
    vs_compound.append(analyzer.polarity_scores(all_data[i]['text'])['compound'])
    vs_pos.append(analyzer.polarity_scores(all_data[i]['text'])['pos'])
    vs_neu.append(analyzer.polarity_scores(all_data[i]['text'])['neu'])
    vs_neg.append(analyzer.polarity_scores(all_data[i]['text'])['neg'])

In [29]:
twitter_df = DataFrame({'Tweet': tweet,
                        'Retweets': number_retweet,
                        'Favourites': number_favourites,
                        'Compound': vs_compound,
                        'Positive': vs_pos,
                        'Neutral': vs_neu,
                        'Negative': vs_neg})

twitter_df = twitter_df[['Tweet','Retweets', 'Favourites', 'Compound',
                         'Positive', 'Neutral', 'Negative']]
twitter_df.sort(columns='Retweets',ascending = False)

# Have a look at the top 5 results.
twitter_df.head()

C:\Users\linj49\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Tweet,Retweets,Favourites,Compound,Positive,Neutral,Negative
0,"trump meant to write ""coverage"". that's it. it...",12017,69382,-0.6492,0.082,0.660,0.258
1,BREAKING: President Trump has decided to withd...,5944,3229,0.0000,0.000,1.000,0.000
2,Trump: My children are having a hard time with...,20223,40715,-0.1027,0.000,0.903,0.097
3,@AnnCoulter They must have graduated from Trum...,0,0,0.0000,0.000,1.000,0.000
4,"U.S. Has Admitted 46,371 Refugees So Far in FY...",0,0,0.6514,0.210,0.790,0.000
